In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# Cluster and Explore Topics

**What**: Convert synthetic abstracts into numerical vectors (TF-IDF) and group them into thematic clusters using K-Means.

**Why**: Clustering allows researchers to discover latent themes in large text corpora without manual labeling. It is a foundational technique for exploratory data analysis.

**How**:
1. **Vectorize text** using TF-IDF to represent documents as numbers.
2. **Apply K-Means clustering** to group similar documents.
3. **Inspect clusters** to interpret the discovered themes.

**Key Concept**: **TF-IDF** (Term Frequency-Inverse Document Frequency) weighs words by how important they are to a document relative to the entire corpus. **K-Means** groups data points into *k* clusters by minimizing the distance to the cluster center.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You built a TF-IDF matrix and k-means clusters.
- You inspected cluster counts.
- You have cluster labels for each abstract.

In [ ]:
from pathlib import Path


def find_data_dir() -> Path:
    candidates = [Path.cwd() / "data", Path.cwd().parent / "data", Path.cwd().parent.parent / "data"]
    for candidate in candidates:
        if (candidate / "sample_texts" / "articles_sample.csv").exists():
            return candidate
    raise FileNotFoundError("data directory not found. Run scripts/generate_synthetic_data.py.")

DATA_DIR = find_data_dir()


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

articles = pd.read_csv(DATA_DIR / "sample_texts" / "articles_sample.csv")
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(articles["abstract"].fillna(""))

model = KMeans(n_clusters=4, random_state=42, n_init=10)
articles["cluster"] = model.fit_predict(tfidf_matrix)
articles[["title", "cluster"]].head()


## Inspect cluster composition

In [ ]:
cluster_counts = articles.groupby("cluster").size().reset_index(name="count")
cluster_counts


### If you get stuck / What to try next

If you get stuck: confirm data generation and rerun the first cell to install dependencies. What to try next: evaluate summaries in pipelines/text/batch_summarization.ipynb or explore retrieval in pipelines/rag/build_index.ipynb.